# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import os
import logging
import boto3
from botocore.exceptions import ClientError
import psycopg2
from helpers.sql import sql_create_tables, staging_insert, sql_drop_tables
from configparser import ConfigParser

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
# Read in the data
directory = os.path.realpath("..") + '/global-land-temp-dw/data/'
tables = ['GlobalLandTemperaturesByMajorCity.csv', 'GlobalLandTemperaturesByCity.csv']

df_major_city = pd.read_csv(directory + tables[0])
df_city = pd.read_csv(directory + tables[1])

In [ ]:
# Drop NA values This data requires all values to be available

def drop_na(pd_df):

    print(f'Number of rows before removing NA: {pd_df.shape[0]:,}')

    pd_df.dropna(axis=0, inplace=True)

    print(f'Number of rows after removing NA: {pd_df.shape[0]:,}')


drop_na(df_major_city)
drop_na(df_city)


In [ ]:
# Add column for Boolean Major city
df_major_city = df_major_city.assign(major_city='true')
df_city = df_city.assign(major_city='false')



In [ ]:
# Concatnate both datasets
dframes = [df_major_city, df_city]
df = pd.concat(dframes, ignore_index=True)

In [ ]:
df.head()

In [ ]:
# Sort data by year
df.sort_values(by='dt',inplace=True)

In [ ]:
df.head(1000)


In [ ]:
# Confirm Data types are as expected
df.dtypes

In [ ]:
# Save data to CSV to load into SQL redshift
df.to_csv(directory + "dataset.csv")

In [ ]:
# Upload to s3 Function from AWS Services https://boto3.amazonaws.com/v1/documentation/api/1.10.46/guide/s3-uploading-files.html

def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [ ]:
# Get Bucket Name from configurations 
config = ConfigParser()
config.read('DW.cfg')
file_log = config.get('S3', 'log_data')

upload_file(directory + "dataset.csv", file_log, "logs/log_data.csv")

#s3 = boto3.client('s3')
#with open(directory + "dataset.csv", "rb") as f:
#    s3.upload_fileobj(f, "global-land-temp", "data_1")

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Star Schema

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [2]:
# Get Database Variables
config = ConfigParser()
config.read('DW.cfg')

print("{}, {}, {}, {}, {}".format(*config['CLUSTER'].values()))


# Connect to redshift database
conn = psycopg2.connect("""host={} 
                           dbname={} 
                           user={} 
                           password={}
                           port={}"""\
                           .format(*config['CLUSTER'].values()    
))

# conn.set_session(autocommit=True)
cur = conn.cursor()


dwhcluster.cg959wyk8kyf.us-west-2.redshift.amazonaws.com, landtempdb, dwhuser, Passw0rd, 5439


In [3]:
# Drop all table if exists
for query in sql_drop_tables:
    cur.execute(query)
    conn.commit()

In [4]:
# Create tables
for query in sql_create_tables:
    cur.execute(query)
    conn.commit()


In [5]:
# Copy data to Staging
sql_query = staging_insert.format(
    's3://global-land-temp/logs/',
    config.get('IAM_ROLE','ARN'),
    config.get('CLUSTER','dwh_region')
    )
print(sql_query)
cur.execute(sql_query)
conn.commit()


    COPY staging_events (
        index,
        dt,
        AverageTemperature,
        AverageTemperatureUncertainty,
        City,
        Country,
        Latitude,
        Longitude,
        major_city
    )
    FROM  's3://global-land-temp/logs/'
    CREDENTIALS 'aws_iam_role=arn:aws:iam::879294216748:role/dwhRole'
    REGION 'eu-west-1'
    DELIMITER ','
    DATEFORMAT 'auto'
    IGNOREHEADER 1
    ;



In [6]:
for query in sql_create_tables:
    print(query)



    CREATE TABLE IF NOT EXISTS cities (
        city_id BIGINT IDENTITY(0,1),
        country_id NUMERIC,
        city TEXT,
        latitude TEXT,
        longitude TEXT
    );


    CREATE TABLE IF NOT EXISTS time (
        dt DATE,
        year SMALLINT,
        month SMALLINT,
        day SMALLINT,
        weekday SMALLINT,
        week SMALLINT
    );


    CREATE TABLE IF NOT EXISTS readings_by_city (
        by_city_id BIGINT IDENTITY(0,1),
        time_id NUMERIC,
        city_id NUMERIC,
        avg_temp NUMERIC,
        avf_temp_uncertainty NUMERIC,
        major_city BOOLEAN
    );


    CREATE TABLE IF NOT EXISTS staging_events (
        index NUMERIC,
        dt DATE NOT NULL,
        AverageTemperature NUMERIC,
        AverageTemperatureUncertainty NUMERIC,
        City TEXT,
        Country TEXT,
        Latitude TEXT,
        Longitude TEXT,
        major_city BOOLEAN
    );



In [ ]:
# Write code here

In [7]:
# Close Connection to the DB
conn.close()

In [ ]:
# Perform quality checks here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.